Figure out what minimum budget you need to get a movie with the best chances of success
Group everything by year
add column with percentage of movie budget vs budget for movies that year
add column with percentage of movie grossing worldwide vs grossing worldwide for movies that year
add column that does a ratio for percent budget and percent grossing. budget % / grossing %. sort lowest to greatest to get movies that did the best with lower budgets.

Take buckets to see what percentage of movies did well with different buckets of budget amounts

In [76]:
#Imports
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt



In [66]:
#prepare and save cleaned data csv
%run clean_data.py

Read in Dataframe and analyze data types and info about structure

In [67]:
#Read in the data
imdb_all_years_df = pd.read_csv("./Resources/cleaned_data.csv")
# imdb_all_years_df.info()
# imdb_all_years_df.head()

#Examples of searching for values
#drop rows where the country of origin list does not contain South Korea
# imdb_south_japan_df = imdb_all_years_df.query("countries_origin.str.contains('Japan')", engine='python')

# #drop rows where the country of origin list does not contain South Korea using loc function
# imdb_south_korea_df = imdb_all_years_df.loc[imdb_all_years_df['countries_origin'].str.contains('Canada', na=False)]

# # imdb_south_japan_df.dropna()

Clean Up Data

In [ ]:
#replace NaN with 0
# imdb_all_years_df = imdb_all_years_df.fillna(1)
# imdb_all_years_df = imdb_all_years_df.dropna(how="any")

#drop all rows that have nan for budget column
budget_df = imdb_all_years_df.dropna(subset=['budget'])
budget_df.info()

In [ ]:
imdb_all_years_df.dtypes

In [ ]:
#sort values by year
imbd_all_years_df = imdb_all_years_df.sort_values(by='Year')
imbd_all_years_df['budget'].dropna(inplace=True)
imbd_all_years_df['Year'].dropna(inplace=True)
imbd_all_years_df['Title'].dropna(inplace=True)
imbd_all_years_df['Year'] = pd.to_numeric(imbd_all_years_df['Year'], errors='coerce')  # Convert to numeric, setting errors as NaN
imbd_all_years_df['Budget'] = pd.to_numeric(imbd_all_years_df['budget'], errors='coerce')  # Same for Budget
imbd_all_years_df
# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(imbd_all_years_df['Year'], imbd_all_years_df['budget'], color='blue')

# Annotate each point with the movie title
# for i, row in imbd_all_years_df.iterrows():
#     plt.text(row['Year'], row['budget'], row['Title'], fontsize=9, ha='right')

plt.xlabel("Year")
plt.ylabel("Budget (in hundred millions)")
plt.title("Movie Budgets Over the Years")
plt.grid(True)
plt.show()

Group and graph data showing worldwide gross of movies over time

In [ ]:
# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(imbd_all_years_df['Year'], imbd_all_years_df['grossWorldWide'], color='blue')

# Annotate each point with the movie title
# for i, row in imbd_all_years_df.iterrows():
#     plt.text(row['Year'], row['budget'], row['Title'], fontsize=9, ha='right')

plt.xlabel("Year")
plt.ylabel("Budget (in billions)")
plt.title("Gross Worldwide Budgets Over the Years")
plt.grid(True)
plt.show()

In [ ]:
#Plot a bar graph with the year on the x-axis and the gross US on the y-axis
plt.figure(figsize=(10, 6))
plt.scatter(imbd_all_years_df['Year'], imbd_all_years_df['gross_US_Canada'], color='blue')

# Annotate each point with the movie title
# for i, row in imbd_all_years_df.iterrows():
#     plt.text(row['Year'], row['budget'], row['Title'], fontsize=9, ha='right')

plt.xlabel("Year")
plt.ylabel("Budget (in hundred millions)")
plt.title("Gross US Canada Budgets Over the Years")
plt.grid(True)
plt.show()

In [ ]:
#Add new column for profit
imdb_all_years_df['Profit'] =  imdb_all_years_df['grossWorldWide'] - imdb_all_years_df['budget']
#Add new column for profit margin
imdb_all_years_df['Profit Margin'] = imdb_all_years_df['Profit'] / imdb_all_years_df['budget']
#add new column for ratio of budget to grossWorldWide
imdb_all_years_df['Budget to Gross Ratio'] = imdb_all_years_df['budget'] / imdb_all_years_df['grossWorldWide']

imdb_all_years_df['Profit']

In [ ]:
bins = np.array([-300000, 0, 100000, 100000000, 500000000])  # Use y values to define colors
bin_colors = ['red', 'blue', 'green', 'purple', 'orange'] 
budget = imbd_all_years_df['budget']
# Assign each y value to a bin
bin_indices = np.digitize(imbd_all_years_df['grossWorldWide'], bins) - 1  # Get bin index for each y value
bin_indices = np.clip(bin_indices, 0, len(bin_colors) - 1)  # Ensure indices are valid

# Normalize budget values for coloring
norm = plt.Normalize(vmin=min(budget), vmax=max(budget))
colormap = plt.cm.coolwarm  # Choose a colormap (e.g., 'plasma', 'coolwarm')

# Map bin indices to colors
colors = [bin_colors[i] for i in bin_indices]

#Plot a bar graph with the year on the x-axis and the gross US on the y-axis
plt.figure(figsize=(10, 6))
plt.scatter(imbd_all_years_df['Year'], imbd_all_years_df['Profit'], c=budget, cmap=colormap, norm=norm)
plt.colorbar()  # Optional: Show color legend

# Annotate each point with the movie title
# for i, row in imbd_all_years_df.iterrows():
#     plt.(row['Year'], row['Profit'], row['Title'], fontsize=9, ha='right',)

plt.xlabel("Year")
plt.ylabel("Profit (in Billions)")
plt.title("Worldwide Profit Over the Years")
plt.grid(True)
plt.show()

In [25]:


bins = np.array([-300000, 0, 100000, 100000000, 500000000])  # Use y values to define colors
bin_colors = ['red', 'blue', 'green', 'purple', 'orange'] 
budget = imdb_all_years_df['budget']
# Assign each y value to a bin
bin_indices = np.digitize(imdb_all_years_df['grossWorldWide'], bins) - 1  # Get bin index for each y value
bin_indices = np.clip(bin_indices, 0, len(bin_colors) - 1)  # Ensure indices are valid

# Normalize budget values for coloring
norm = plt.Normalize(vmin=min(budget), vmax=max(budget))
colormap = plt.cm.coolwarm  # Choose a colormap (e.g., 'plasma', 'coolwarm')

# Map bin indices to colors
colors = [bin_colors[i] for i in bin_indices]

#Plot a bar graph with the year on the x-axis and the gross US on the y-axis
plt.figure(figsize=(10, 6))
plt.scatter(imdb_all_years_df['Year'], imdb_all_years_df['Profit'], c=budget, cmap=colormap, norm=norm)
plt.colorbar()  # Optional: Show color legend

# Annotate each point with the movie title
# for i, row in imbd_all_years_df.iterrows():
#     plt.(row['Year'], row['Profit'], row['Title'], fontsize=9, ha='right',)

plt.xlabel("Year")
plt.ylabel("Profit (in Billions)")
plt.title("Worldwide Profit Over the Years for movies profiting over 5 mil")
plt.grid(True)
plt.show()

In [ ]:


# #Plot a bar graph with the year on the x-axis and the budget to grossworldwide on the y-axis
# plt.title("Average Budget/GrossWorldWide by Year")
# plt.xlabel('Year')
# plt.ylabel('Avg. Budget/GrossWorldWide')
# plt.bar(imdb_all_years_df['Year'], imdb_all_years_df['Budget to Gross Ratio'])

In [ ]:
#Its Clear from the above graph that movies on average dont make much money until recently.
# imdb_all_years_df.sort_values(by='Budget to Gross Ratio', ascending=True)